In [ ]:
%load_ext autoreload
%autoreload 2

# PySerialEM - montaging

https://github.com/instamatic-dev/pyserialem

This notebook shows how to process a grid montage acquired using `SerialEM`. The data for this demo were collected on a zeolite sample (2020-02-12), using a JEOL JEM-1400 @ 120 kV in combination with a TVIPS F-416 camera. 

The data are available from zenodo: https://doi.org/10.5281/zenodo.3923718

These data were chosen, because the stitching from SerialEM was particularly bad. We will show an example of how `PySerialEM` can be used to try to get a better montage.

For this demo to work, change the `work` directory below to point at the right location.

In [ ]:
from pyserialem import Montage
import numpy as np
from pathlib import Path
np.set_printoptions(suppress=True)

# work directory
work = Path(r"C:\s\2020-02-12\serialem_montage")
work

## Setting up the montage

Load the `gm.mrc` file and the associated images. For SerialEM data, the gridshape must be specified, because it cannot be obtained from the data or `.mdoc` direction. There are also several parameters to tune the orientation of the images to match them with the input of the stagematrix (if needed). First we must get the coordinate settings to match those of SerialEM. These variables ap

In [ ]:
m = Montage.from_serialem_mrc(
    work / 'gm.mrc', 
    gridshape=(5,5),
    direction='updown',
    zigzag=True,
    flip=False,
    image_rot90 = 3,
    image_flipud = False,
    image_fliplr = True,
)

m.gridspec

First, we can check what the data actually look like. To do so, we can simply `stitch` and `plot` the data using a `binning=4` to conserve a bit of memory. This naively plots the data at the expected positions. Although the stitching is not that great, it's enough to get a feeling for the data.

Note that SerialEM includes the pixel coordinates in the `.mdoc` file, so it is not necessary to calculate these again. Instead, the `PieceCoordinates` are mapped to `m.coords`.

In [ ]:
# Use `optimized = False` to prevent using the aligned piece coordinates
m.stitch(binning=4, optimized=False)
m.plot()

SerialEM has also already calculated the aligned image coordinates (`AlignedPieceCoordsVS`/`AlignedPieceCoords`). These can be accessed via the `.optimized_coords` attribute. To plot, you can do the following:

In [ ]:
# optimized = True is the default, so it can be left out
m.stitch(binning=4, optimized=True)
m.plot()

montage_serialem = m.stitched  # store reference for later

The stitching from SerialEM is particularly bad, so we can try to optimize it using the algorithm in `pyserialem`.
First, we must ensure that we entered the gridspec correctly. If the layout of the tiles below does not look right (i.e. similar to above), go back to loading the `Montage` and fiddle with the rotation of the images. The operation below just places the tiles at the positions calculated by `pyserialem`.

In [ ]:
m.calculate_montage_coords()
m.stitch(binning=4, optimized=False)
m.plot()

It is still possible to try to get better stitching using the algorithm in `pyserialem`:

 1. Better estimate the difference vectors between each tile using cross correlation
 2. Optimize the coordinates of the difference vectors using least-squares minimization

This approach is based on *Globally optimal stitching of tiled 3D microscopic image acquisitions* by Preibish et al., Bioinformatics 25 (2009), 1463–1465 (https://doi.org/10.1093/bioinformatics/btp184).

Some metrics, such as the obtained shifts and FFT scores are plotted to evaluate the stitching.

In [ ]:
# Use cross correlation to get difference vectors
m.calculate_difference_vectors(
    threshold=0.08, 
#     method='skimage',
    plot=False
)

# plot the fft_scores
m.plot_fft_scores()

# plot the pixel shifts
m.plot_shifts()

# get coords optimized using cross correlation
m.optimize_montage_coords(plot=True)

# stitch image, use binning 4 for speed-up and memory conservation
m.stitch(binning=4)

# plot the stitched image
m.plot()

montage_pyserialem = m.stitched  # store reference for later

It's not a perfect stitching, but much better than what SerialEM produced! I believe the reason is that SerialEM does some adjustments to the stageposition as it is moving. Below is an example of the same grid map collected with [instamatic](http://github.com/instamatic-dev/instamatic), using the same coordinates and imaging conditions, reconstructed with the same algorithm.

In [ ]:
import tifffile
import matplotlib.pyplot as plt

with tifffile.TiffFile(work / 'stitched_instamatic.tiff') as f:
    montage_instamatic = f.asarray()

In [ ]:
fig, (ax0, ax1, ax2) = plt.subplots(ncols=3, figsize=(20,10))

ax0.imshow(montage_serialem)
ax0.set_title('Data: SerialEM\n'
              'Stitching: SerialEM')

ax1.imshow(montage_pyserialem)
ax1.set_title('Data: SerialEM\n'
              'Stitching: PySerialEM')

ax2.imshow(montage_instamatic)
ax2.set_title('Data: Instamatic\n'
              'Stitching: PySerialEM');

When the image has been stitched (with or without optimization), we can look for the positions of the grid squares/squircles. 

First, we should tell `pyserialem` about the imaging conditions by setting the stagematrix to relate the pixelpositions back to stage coordinates. 

The easiest way to do it is to pass the `StageToCameraMatrix` directly. It can be found in `SerialEMcalibrations.txt`. Look for the last number, which gives the magnification.

(They can also be set directly via `.set_stagematrix` and `.set_pixelsize`)

In [ ]:
StageToCameraMatrix = "StageToCameraMatrix 10 0 8.797544 0.052175 0.239726 8.460119   0.741238   100"
m.set_stagematrix_from_serialem_calib(StageToCameraMatrix)

m.stagematrix

This also sets the pixelsize.

In [ ]:
m.pixelsize

To find the holes, call the method `.find_holes`. The grid squares are identified as objects roughly sized `diameter` with a tolerance of 10%. The median as well as 5/95 percentiles are printed to evaluate the hole size distribution. By default the `otsu` method is used to define the threshold, but the threshold can be changed if the segmentation looks poor.

In [ ]:
stagecoords, imagecoords = m.find_holes(
    plot=True, 
    tolerance=0.2)

IF a `.nav` file was saved, the stage coordinates can be added and read back into `SerialEM`.

In [ ]:
from pyserialem import read_nav_file, write_nav_file

nav = read_nav_file(work / "nav.nav")
map_item = nav[0]
items = map_item.add_marker_group(coords=stagecoords/1000, kind='stage')
write_nav_file("nav_new.nav", map_item, *items)

It is possible to optimize the stage coordinates for more efficient navigation. In this example, the total stage movement can be reduced by about 75%, which will save a lot of time. The function uses the _two-opt_ algorithm for finding the shortest path: https://en.wikipedia.org/wiki/2-opt.

In [ ]:
from pyserialem.navigation import sort_nav_items_by_shortest_path

stagecoords = sort_nav_items_by_shortest_path(
    stagecoords, 
    plot=True
)

We can re-run the command (or set the `threshold` to something like `0.01`) to try to get a better path. In this case it's possible to improve it a little bit more.

In [ ]:
stagecoords = sort_nav_items_by_shortest_path(
    stagecoords, 
    threshold=0.01,
    plot=True
)